<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Tinoco
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** https://github.com/sebatinoco/MDS7202/tree/main/lab9`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install praw
!pip install line_profiler
%load_ext line_profiler
!pip install memory_profiler # instalamos memory profiler
%load_ext memory_profiler
!pip install numba

%load_ext autoreload
%autoreload 2

import os
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [3]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot)) # obtenemos el top n de posts del foro
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n] # obtiene los primeros 50 posts
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: # para cada uno de los posts
            try:
                ratio = submission.upvote_ratio # obtenemos el ratio de upvote
                ups = int( # calculamos upvotes
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups) # guardamos upvotes
                downvotes.append(ups - submission.score) # guardamos downvotes
                post[it].append(submission.title) # guardamos titulo del post
                url[it].append(submission.url) # guardamos url
            except Exception as e: # en caso de error, sigue 
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url # retorna votos, post y url

In [4]:
votes, post, url = praw_reddit()

El propósito del código adjunto es obtener métricas (*upvotes*, *downvotes*, *título* y *url*) de posts en el subreddit de "chile". En general, la implementación sigue los siguientes pasos:

1. Instanciar la clase `reddit` en el subreddit `nombre_subreddit`
2. Obtener los primeros `n_hot` posts del subreddit especificado
3. En batchs de 50, obtener *upvotes*, *downvotes*, *título* y *url* de cada uno de los posts y luego los guarda en una listas dentro de un diccionario (en el caso de *upvotes* y *downvotes*, estos son guardados en np.arrays dentro del diccionario).
4. Finalmente, la función devuelve los diccionarios generados.

In [5]:
# estudio de tiempos de ejecución
%lprun -f praw_reddit praw_reddit()

In [6]:
%memit praw_reddit()

peak memory: 258.73 MiB, increment: 0.95 MiB


In [7]:
%%file praw_module.py
import numpy as np
import praw
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot)) # obtenemos el top n de posts del foro
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n] # obtiene los primeros 50 posts
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: # para cada uno de los posts
            try:
                ratio = submission.upvote_ratio # obtenemos el ratio de upvote
                ups = int( # calculamos upvotes
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups) # guardamos upvotes
                downvotes.append(ups - submission.score) # guardamos downvotes
                post[it].append(submission.title) # guardamos titulo del post
                url[it].append(submission.url) # guardamos url
            except Exception as e: # en caso de error, sigue 
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url # retorna votos, post y url

Overwriting praw_module.py


In [8]:
from praw_module import praw_reddit 

%mprun -f praw_reddit praw_reddit()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



**Respuesta:**

Notamos que los tiempos de ejecución de la función están totalmente cargados hacia la línea `top_submissions = list(subreddit.hot(limit=n_hot))`, tomando el 99.4% del tiempo total de ejecución. El resto del tiempo se lo toma en su mayoría los *append* que se hacen para guardar los valores en las listas. Así, es esperable que la mayor cantidad del tiempo sea destinado a obtener los posts del foro, pues implica obtener hacer una "query" a la API de reddit, esperar que sea procesada y luego obtener la respuesta, generando automáticamente un tiempo de espera mayor.

Finalmente, notamos que casi la totalidad de la memoria de la función está siendo usada en la línea `def praw_reddit(nombre_subreddit="chile", n_hot=1000)`, es decir, en la definción de la función `praw_reddit`. De esta forma, como definir una función global implica que tenga que ser guardada en el *runtime* de Python y pueda ser llamada en cualquier momento, es natural que sea la línea con uso de memoria.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

1. El cache es un espacio de memoria almacenado en la memoria RAM, disco duro o de manera remota con el fin de hacer **memoización**, es decir, acceder a resultados previamente calculados guardándolos en cache, resultando en tiempos notablemente menores de ejecución. Por otro lado, las siglas LRU vienen de "Least Recently Used".

2. El principal costo de usar cache es aumentar el uso de memoria, aumentando directamente el costo computacional del código. Además, en caso de usar cache almacenado en disco, el acceso a los resultados almacenados puede ser muy lento, afectando negativamente al rendimiento del código.

3. Finalmente, usar cache en la función anterior tiene diversas implicancias: En primer lugar, los resultados guardados en memoria serían estáticos, es decir, se guardarían en cache los posts de reddit que fueron resultado de la primera invocación de la función, por lo que invocar nuevamente la función traería esos mismos resultados y no actualizaría la información con los nuevos registros. En segundo lugar y como ya vimos, usar cache en la función disminuiría notablemente el tiempo de ejecución de la función. Finalmente, usar cache implicaría un aumento de uso de la memoria. 

In [9]:
# aprovechamos que ya habiamos definido la función sin caché
%%timeit 
praw_reddit()

1 loop, best of 5: 20.9 s per loop


In [11]:
@lru_cache()
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot)) # obtenemos el top n de posts del foro
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n] # obtiene los primeros 50 posts
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions: # para cada uno de los posts
            try:
                ratio = submission.upvote_ratio # obtenemos el ratio de upvote
                ups = int( # calculamos upvotes
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups) # guardamos upvotes
                downvotes.append(ups - submission.score) # guardamos downvotes
                post[it].append(submission.title) # guardamos titulo del post
                url[it].append(submission.url) # guardamos url
            except Exception as e: # en caso de error, sigue 
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url # retorna votos, post y url

In [12]:
%%timeit 
votes, post, url = praw_reddit()

The slowest run took 111006872.26 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 184 ns per loop


**Respuesta de lru_cache sobre praw_reddit**

El tiempo promedio por loop de la función sin cache es de 20.9 segundos, mientras que el  tiempo de ejecución total de este ejercicio asciende a 128 segundos. Por otro lado, usar la función con cache lleva a que el tiempo promedio por loop de ejecución sea de 184 nanosegundos, mientras que el tiempo total de ejecución es de 20.5 segundos. Un aspecto importante acá es que la primera invocación a la función se lleva casi todo el tiempo de ejecución, demorando una magnitud bastaaaante superior que la ejecución más rápida. Esto último se debe a que en la primera ejecución la función aun no tiene cache, situación que cambia desde la segunda ejecución en adelante. Finalmente y en vista de los resultados, es posible concluir que se visualizan notables mejoras en cuanto a los tiempos de ejecución.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [14]:
def fun1(votes):
  '''
  Una función muy mal optimizada.
  '''

  mu = []
  sigma = []
  for row in votes:
    u = row[0]
    d = row[1]

    a = 1 + u
    b = 1 + d 

    mu.append(a/(a+b) )
    sigma.append(1.65 * ((a*b)/(((a+b)**2) * (a+b+1)))**(1/2))

  return mu, sigma

In [15]:
def fun2(votes):
  '''
  Una función un poco mejor optimizada.
  '''

  a = np.ones(1) + votes[:, 0]
  b = np.ones(1) + votes[:, 1]

  mu = np.divide(a, a+b)
  sigma = 1.65 * np.sqrt(np.divide(a*b, np.multiply((a+b)**2, a+b+1)))

  return mu, sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [16]:
from numba import jit

@jit(nopython=True)
def fun1_numba(votes):
  '''
  Una función muy mal optimizada.
  '''

  mu = []
  sigma = []
  for row in votes:
    u = row[0]
    d = row[1]

    a = 1 + u
    b = 1 + d 

    mu.append(a/(a+b) )
    sigma.append(1.65 * ((a*b)/(((a+b)**2) * (a+b+1)))**(1/2))

  return mu, sigma

@jit(nopython=True)
def fun2_numba(votes):
  '''
  Una función un poco mejor optimizada.
  '''

  a = np.ones(1) + votes[:, 0]
  b = np.ones(1) + votes[:, 1]

  mu = np.divide(a, a+b)
  sigma = 1.65 * np.sqrt(np.divide(a*b, np.multiply((a+b)**2, a+b+1)))

  return mu, sigma

# ejecutamos ambas funciones por primera vez
fun1_numba(votes[1])
fun2_numba(votes[1])

(array([0.53977273, 0.82706767, 0.94444444, 0.87096774, 0.96363636,
        0.93421053, 0.9122807 , 0.92941176, 0.8627451 , 0.95238095,
        0.84848485, 0.83636364, 0.875     , 0.87434555, 0.95      ,
        0.94761905, 0.79166667, 0.97935103, 0.80952381, 0.97594502,
        0.88888889, 0.84615385, 0.79166667, 0.59821429, 0.88571429,
        0.76      , 0.94520548, 0.82857143, 0.76470588, 0.8       ,
        0.95535714, 0.98879552, 0.88861985, 0.90807799, 0.66666667,
        0.66666667, 0.91919192, 0.9047619 , 0.81081081, 0.93820225,
        0.98039216, 0.71612903, 0.5625    , 0.8       , 0.73333333,
        0.88679245, 0.578125  , 0.77380952, 0.58333333, 0.95454545,
        0.91752577, 0.625     , 0.5       , 0.5       , 0.93939394,
        0.64615385, 0.86      , 0.90784983, 0.81578947, 0.7826087 ,
        0.93693694, 0.5       , 0.86813187, 0.8943662 , 0.79411765,
        0.94      , 0.86842105, 0.57894737, 0.79411765, 0.87179487,
        0.80392157, 0.75862069, 0.95061728, 0.53

Luego, generamos la función `return_time` para devolver el tiempo de ejecución para cada función. Nos aprovecharemos de que `votes` tiene batches de diferente tamaño (batches de tamaño 50, 100, 150, etc..) para ir graficando a medida que aumenta el tamaño del batch size

In [17]:
def return_time(func, batch):

  '''
  Función que recibe una función de cálculo y un batch de tamaño n, devuelve el tiempo de computación.
  func: función que calcula mean posterior y std
  batch: batch de datos a calcular
  '''

  t0 = time.process_time() # tiempo inicial
  func(batch)
  t1 = time.process_time()
  return t1 - t0

In [18]:
# obtenemos tiempos de ejecución para cada función
fun1_times, fun2_times, fun1_numba_times, fun2_numba_times = [], [], [], []
for it, batch in votes.items():
  fun1_times.append(return_time(fun1, batch))
  fun2_times.append(return_time(fun2, batch))
  fun1_numba_times.append(return_time(fun1_numba, batch))
  fun2_numba_times.append(return_time(fun2_numba, batch))

# generamos dataframe y melt para graficar
df_times = pd.DataFrame({'Python': fun1_times, 'Numpy': fun2_times, 'Python-JIT': fun1_numba_times, 'Numpy-JIT': fun2_numba_times})
df_times = df_times.melt(var_name = 'tipo', value_name = 'Tiempo')
df_times['it'] = (df_times.groupby('tipo').cumcount() + 1) * 50

# graficamos usando plotly
fig = px.line(df_times, x = 'it', y = 'Tiempo', color = 'tipo', log_y = True)
fig.show()

En vista del gráfico, es posible visualizar una notoria diferencia en los tiempso de ejecución de las funciones: 

1. En primer lugar, notamos que la función hecha en Python obtiene por lejos el peor rendimiento, alcanzo tiempos de ejecución bastante superiores que las demás implementaciones. Esto es esperable pues se está usando código python nativo y no se usa ningún tipo de optimización de código.

2. En segundo lugar, notamos que las implementaciones Numpy y Python-JIT obtienen rendimientos bastante similares. Esto es interesante, pues tanto Numpy como JIT optimizan el código de alguna forma: mientras que Numpy lo hace a través de la vectorización de las operaciones, JIT optimiza el código usando compilación de código, moviendo el código de alto nivel a lenguaje de máquina y por lo tanto disminuyendo considerablemente los tiempos de ejecución.

3. Finalmente, se observa que la combinación de Numpy y JIT es la más rentable en términos de tiempos de ejecución, superando con creces el rendimiento de las otras implementaciones. La razón de su rendimiento se debe a que se aprovecha la optimización tanto de Numpy (operaciones vectoriales) como de JIT (compilación de código), logrando excelentes resultados.

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [19]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
#posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1]) # comentamos esta función pues habia que remplazarla por la nuestra :p
posterior_mean, std_err = fun2_numba(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [20]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

**Respuesta:**

Analizando los resultados del gráfico, notamos que, a pesar de que los posts estan ordenados por el límite inferior, sus medias no estan necesariamente ordenadas. Lo anterior implica que si bien un post puede tener un límite inferior menor que el límite inferior de otro post, esto no implica que el ratio $\frac{upvotes}{total\_votes}$ sigan el mismo orden. En ese sentido, la gran conclusión es que para concluir que un post es bueno o malo se debe considerar el análisis tanto de la media como de la desviación estándar y no cada una por sí sola, resultando en un análisis no trivial.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>